# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Adamstown,-25.0660,-130.1015,72.09,87,98,28.10,PN,1691513707
1,1,Waitangi,-43.9535,-176.5597,47.82,96,100,1.99,NZ,1691513707
2,2,Yaren,-0.5472,166.9160,82.40,72,93,7.87,NR,1691513707
3,3,Whakatane,-37.9585,176.9854,39.78,99,16,5.91,NZ,1691513707
4,4,Port Alfred,-33.5906,26.8910,61.18,86,96,4.00,ZA,1691513707


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "Humidity",

) 


# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = pd.DataFrame (city_data_df, columns = ["City", "Cloudiness", "Humidity", "Wind Speed", "Max Temp"])
ideal_city_df = ideal_city_df.loc[(ideal_city_df["Cloudiness"] == 0) & (ideal_city_df["Humidity"] < 50) & (ideal_city_df["Wind Speed"] < 5)]

# Drop any rows with null values
clean_ideal_city_df = ideal_city_df.dropna()

# Display sample data
clean_ideal_city_df

,City,Cloudiness,Humidity,Wind Speed,Max Temp
54,Diamantino,0,16,2.53,104.00
85,Robbah,0,15,3.09,103.64
161,Debila,0,17,4.61,104.07
257,Madera,0,28,4.65,92.25
329,Galeana,0,27,1.90,81.63
365,Chowchilla,0,41,3.44,85.01
419,Newman,0,29,3.15,83.23
548,Torre de Moncorvo Municipality,0,17,1.99,104.70


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame (city_data_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Adamstown,PN,-25.0660,-130.1015,87,
1,Waitangi,NZ,-43.9535,-176.5597,96,
2,Yaren,NR,-0.5472,166.9160,72,
3,Whakatane,NZ,-37.9585,176.9854,99,
4,Port Alfred,ZA,-33.5906,26.8910,86,
...,...,...,...,...,...,...
595,Knyaze-Volkonskoye,RU,48.4642,135.4567,68,
596,Piaçabuçu,BR,-10.4056,-36.4344,80,
597,Tver',RU,56.8619,35.8931,83,
598,Bikenibeu Village,KI,1.3673,173.1241,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
limit = 20
radius = 10000

params = {"catagories" : "accomodation:hotel",
          "api_key": geoapify_key,
          "limit": limit}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   
    # get latitude, longitude from the DataFrame
    
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["filters"] = f"circle:{Lng},{Lat}, {radius}"
    params["bias"] = f"proximity:{Lng},{Lat}" 
    
   
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    name_address = response.json()    
    print(params)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address[0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-130.1015,-25.066, 10000', 'bias': 'proximity:-130.1015,-25.066'}
Adamstown - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-176.5597,-43.9535, 10000', 'bias': 'proximity:-176.5597,-43.9535'}
Waitangi - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:166.916,-0.5472, 10000', 'bias': 'proximity:166.916,-0.5472'}
Yaren - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:176.9854,-37.9585, 10000', 'bias': 'proximity:176.9854,-37.9585'}
Whakatane - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limi

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-8.4694,54.2697, 10000', 'bias': 'proximity:-8.4694,54.2697'}
Sligo - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:104.0439,20.4197, 10000', 'bias': 'proximity:104.0439,20.4197'}
Xam Nua - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:0.4736,7.1518, 10000', 'bias': 'proximity:0.4736,7.1518'}
Hohoe - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-114.3525,62.456, 10000', 'bias': 'proximity:-114.3525,62.456'}
Yellowknife - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-8.7187,70.9

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:15.2658,40.2297, 10000', 'bias': 'proximity:15.2658,40.2297'}
Vallo della Lucania - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-62.2667,-13.0833, 10000', 'bias': 'proximity:-62.2667,-13.0833'}
Pôsto Fiscal Rolim de Moura - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:100.3354,5.4112, 10000', 'bias': 'proximity:100.3354,5.4112'}
George Town - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-60.3315,53.3168, 10000', 'bias': 'proximity:-60.3315,53.3168'}
Happy Valley-Goose Bay - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb96060

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:169.8474,11.2278, 10000', 'bias': 'proximity:169.8474,11.2278'}
Utrik - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:33.6111,1.7146, 10000', 'bias': 'proximity:33.6111,1.7146'}
Soroti - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-104.7125,47.1053, 10000', 'bias': 'proximity:-104.7125,47.1053'}
Glendive - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-118.0467,33.8464, 10000', 'bias': 'proximity:-118.0467,33.8464'}
La Palma - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:172.6

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-157.9253,21.6477, 10000', 'bias': 'proximity:-157.9253,21.6477'}
Lā‘ie - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:150.8,59.5667, 10000', 'bias': 'proximity:150.8,59.5667'}
Magadan - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-90.3518,-0.7393, 10000', 'bias': 'proximity:-90.3518,-0.7393'}
Puerto Ayora - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:49.2917,-12.2787, 10000', 'bias': 'proximity:49.2917,-12.2787'}
Antsiranana - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-

...

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-149.55,-17.75, 10000', 'bias': 'proximity:-149.55,-17.75'}
Papao - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:13.1092,14.2495, 10000', 'bias': 'proximity:13.1092,14.2495'}
Nguigmi - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:95.3214,5.8933, 10000', 'bias': 'proximity:95.3214,5.8933'}
Sabang - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:106.2542,29.2906, 10000', 'bias': 'proximity:106.2542,29.2906'}
Jijiang - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:115.1723,-8.7233,

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-3.6104,14.3501, 10000', 'bias': 'proximity:-3.6104,14.3501'}
Bandiagara - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:88.3103,22.5892, 10000', 'bias': 'proximity:88.3103,22.5892'}
Howrah - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:25.9931,64.808, 10000', 'bias': 'proximity:25.9931,64.808'}
Muhos - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:128.9167,-3.3333, 10000', 'bias': 'proximity:128.9167,-3.3333'}
Amahai - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-43.0256,-7.6

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-5.7934,43.6152, 10000', 'bias': 'proximity:-5.7934,43.6152'}
Luanco - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:30.3768,46.0272, 10000', 'bias': 'proximity:30.3768,46.0272'}
Serhiyivka - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:50.8122,9.4942, 10000', 'bias': 'proximity:50.8122,9.4942'}
Bandarbeyla - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:121.4233,13.9311, 10000', 'bias': 'proximity:121.4233,13.9311'}
Candelaria - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:18.

{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-2.0981,57.1437, 10000', 'bias': 'proximity:-2.0981,57.1437'}
Aberdeen - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:-88.569,47.1219, 10000', 'bias': 'proximity:-88.569,47.1219'}
Houghton - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:122.0833,46.0833, 10000', 'bias': 'proximity:122.0833,46.0833'}
Ulanhot - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:30.4833,19.1667, 10000', 'bias': 'proximity:30.4833,19.1667'}
Dongola - nearest hotel: No hotel found
{'catagories': 'accomodation:hotel', 'api_key': 'f2778a6658bb441cb960601d310dc324', 'limit': 20, 'filters': 'circle:11.0144,

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Adamstown,PN,-25.0660,-130.1015,87,No hotel found
1,Waitangi,NZ,-43.9535,-176.5597,96,No hotel found
2,Yaren,NR,-0.5472,166.9160,72,No hotel found
3,Whakatane,NZ,-37.9585,176.9854,99,No hotel found
4,Port Alfred,ZA,-33.5906,26.8910,86,No hotel found
...,...,...,...,...,...,...
595,Knyaze-Volkonskoye,RU,48.4642,135.4567,68,No hotel found
596,Piaçabuçu,BR,-10.4056,-36.4344,80,No hotel found
597,Tver',RU,56.8619,35.8931,83,No hotel found
598,Bikenibeu Village,KI,1.3673,173.1241,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "Hotel Name",

) 


# Display the map plot
map_plot_1